In [1]:
from pathlib import Path
from conllu import parse
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

notebook_path = Path().resolve()
code_folder = notebook_path.parent
treebanks_folder = code_folder / "treebanks"
print(treebanks_folder)

nltk.download('stopwords')
nltk.download('punkt')
norwegian_stop_words = set(stopwords.words("norwegian"))

/Users/liamo/Desktop/School/550/FinalProj/comp550-norwegian-dialects/treebanks


[nltk_data] Downloading package stopwords to /Users/liamo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/liamo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def parse_conll_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        conll_data = file.read()
        return parse(conll_data)

In [3]:
#In this function we will have a nested list all_parsed_data (nested list of sentences where the outter list is a specific dialect while the inner list is the sentences per dialect)
#The second list all_parsed_data_filtered is the filtered version of the same thing
#Each element in the inner list is a token and there is meta data at the end
#Each token has associated info lemma,upos,etc
all_parsed_data = [] 
print(treebanks_folder)

for file_path in treebanks_folder.glob("*.conll"):
    parsed_data = parse_conll_file(file_path)
    all_parsed_data.append(parsed_data)


if all_parsed_data:
    first_file_parsed_data = all_parsed_data[0][1]
    serialized = first_file_parsed_data.serialize()
    print(first_file_parsed_data)
    print(serialized)
    token = first_file_parsed_data[0]
    # print(token['id'])
    # print(token['form'])
    # print(token['lemma'])
    # print(token['upos'])
else:
    all_parsed_data = []

/Users/liamo/Desktop/School/550/FinalProj/comp550-norwegian-dialects/treebanks
TokenList<frå, #, Førde, ?, metadata={text: "frå # Førde ?", segstart: "10.891", segstop: "11.983", file: "foerde_uib_05", speaker: "int1", id: "2"}>
# text = frå # Førde ?
# segstart = 10.891
# segstop = 11.983
# file = foerde_uib_05
# speaker = int1
# id = 2
1	frå	frå	prep	prep	_	0	FRAG	_	_
2	#	#	pause	pause	_	3	IK	_	_
3	Førde	Førde	subst	subst	prop	1	PUTFYLL	_	_
4	?	$?	clb	clb	<spm>	1	IP	_	_




In [19]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

#code referenced here https://github.com/christianversloot/machine-learning-articles/blob/main/bidirectional-lstms-with-tensorflow-and-keras.md
additional_metrics = ['accuracy']
batch_size = 128
embedding_output_dims = 70
loss_function = CategoricalCrossentropy()
max_sequence_length = 70
num_distinct_words = 10000
number_of_epochs = 100
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

In [20]:
word_2_int = {}
i = 2
for dialect in all_parsed_data:
    for token_list in dialect:
        sentence = token_list.metadata.get('text', '').split()
        for word in sentence:
            if word not in word_2_int:
                word_2_int[word] = i
                i += 1

In [22]:
better_data ={}

for dialect in all_parsed_data:

    dialect_name = dialect[0].metadata.get('file', '')
    better_data[dialect_name] = dialect

ls_aal = better_data['aal_uio_02'] + better_data['aal_uio_06']
ls_austevoll = better_data['austevoll_uib_01'] + better_data['austevoll_uib_04']

better_data['aal_uio_02'] = ls_aal
better_data['austevoll_uib_01'] = ls_austevoll

del better_data['aal_uio_06']
del better_data['austevoll_uib_04']

temp_data = {}
for dialect, token_list in better_data.items():
    temp_ls = []
    for token in token_list:
        sentence = token.metadata.get('text', '')
        if len(token) > 3:
            temp_ls.append(sentence)
    temp_data[dialect] = temp_ls

In [23]:
integer_encoded = {}
for key,values in better_data.items():
    for token_list in values:
        if key not in integer_encoded:
            integer_encoded[key] = []
        
        sentence = token_list.metadata.get('text', '').split()
        new_sentence = [1]
        for word in sentence:
            new_sentence.append(word_2_int[word])
        integer_encoded[key].append(new_sentence)

In [24]:
def prepare_data(num_classes, integer_encoded):
    from sklearn.model_selection import train_test_split
    #split into train and test
    x_train = []
    x_test = []

    y_train = []
    y_test = []

    if num_classes == 3:   
        dialect_mapping ={
            'flakstad_uib_04': 0,
            'giske_uib_02': 1,
            'eidsberg_uio_03': 2,
        }

    elif num_classes == 6:
        dialect_mapping = {
            'flakstad_uib_04': 0,
            'giske_uib_02': 1,
            'eidsberg_uio_03': 2,
            'austevoll_uib_01': 3,
            'bardu_uit_01': 4,
            'vardoe_uio_01': 5
        }

    elif num_classes == 12:
        dialect_mapping = {
            'flakstad_uib_04': 0,
            'giske_uib_02': 1,
            'eidsberg_uio_03': 2,
            'austevoll_uib_01': 3,
            'bardu_uit_01': 4,
            'vardoe_uio_01': 5,
            'aal_uio_02': 6,
            'brandbu_uio_01_ny': 7,
            'hjartdal_uio_01': 8,
            'farsund_uib_02': 9,
            'foerde_uib_05': 10,
            'lierne_uio_01': 11
        }

    elif num_classes == 17:
        dialect_mapping = {
            'flakstad_uib_04': 0,
            'giske_uib_02': 1,
            'eidsberg_uio_03': 2,
            'austevoll_uib_01': 3,
            'bardu_uit_01': 4,
            'vardoe_uio_01': 5,
            'aal_uio_02': 6,
            'brandbu_uio_01_ny': 7,
            'hjartdal_uio_01': 8,
            'farsund_uib_02': 9,
            'foerde_uib_05': 10,
            'lierne_uio_01': 11,
            'fana_uib_03': 12,
            'hemsedal_uio_01': 13,
            'herad_uio_01': 14,
            'gol_uio_01': 15,
            'hoeyanger_uib_02': 16
        }

    integer_encoded = {key: integer_encoded[key] for key in dialect_mapping if key in integer_encoded}

    for key, values in integer_encoded.items():
        train_data, test_data = train_test_split(values, test_size=0.2, random_state=42)
        x_train.extend(train_data)
        x_test.extend(test_data)
        y_train.extend([dialect_mapping[key]] * len(train_data))
        y_test.extend([dialect_mapping[key]] * len(test_data))
    
    x_train = pad_sequences(x_train, maxlen=70, value = 0.0)
    x_test = pad_sequences(x_test, maxlen=70, value = 0.0)

    from keras.utils import to_categorical

    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return x_train, x_test, np.array(y_train), np.array(y_test)


In [25]:
def test_model(num_classes,x_train,x_test,y_train,y_test):
    #code referenced here: https://github.com/christianversloot/machine-learning-articles/blob/main/bidirectional-lstms-with-tensorflow-and-keras.md
    model = Sequential()
    model.add(Embedding(num_distinct_words, embedding_output_dims, input_length=max_sequence_length))
    model.add(Bidirectional(LSTM(70)))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=additional_metrics)

    model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=number_of_epochs,
            verbose=verbosity_mode,
            validation_split=validation_split)

    test_results = model.evaluate(x_test, y_test, verbose=0)
    print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
    return model

In [12]:
x_train_3, x_test_3, y_train_3, y_test_3 = prepare_data(3, integer_encoded)
model_3 = test_model(3,x_train_3, x_test_3, y_train_3, y_test_3)
#2.18

Epoch 1/100
15/15 [==============================] - 3s 115ms/step - loss: 0.9626 - accuracy: 0.5554 - val_loss: 3.1930 - val_accuracy: 0.0000e+00
Epoch 2/100
15/15 [==============================] - 1s 88ms/step - loss: 0.8287 - accuracy: 0.5831 - val_loss: 2.9862 - val_accuracy: 0.0000e+00
Epoch 3/100
15/15 [==============================] - 1s 82ms/step - loss: 0.7638 - accuracy: 0.6467 - val_loss: 2.8725 - val_accuracy: 0.0000e+00
Epoch 4/100
15/15 [==============================] - 1s 85ms/step - loss: 0.6401 - accuracy: 0.7558 - val_loss: 2.6888 - val_accuracy: 0.0000e+00
Epoch 5/100
15/15 [==============================] - 1s 85ms/step - loss: 0.4891 - accuracy: 0.8189 - val_loss: 2.7031 - val_accuracy: 0.0000e+00
Epoch 6/100
15/15 [==============================] - 1s 89ms/step - loss: 0.3998 - accuracy: 0.8538 - val_loss: 2.9503 - val_accuracy: 0.0000e+00
Epoch 7/100
15/15 [==============================] - 1s 81ms/step - loss: 0.3448 - accuracy: 0.8682 - val_loss: 2.5922 - va

In [13]:
x_train_6, x_test_6, y_train_6, y_test_6 = prepare_data(6, integer_encoded)
model_6 = test_model(6,x_train_6, x_test_6, y_train_6, y_test_6)
#3.16

Epoch 1/100
23/23 [==============================] - 3s 92ms/step - loss: 1.5942 - accuracy: 0.3253 - val_loss: 4.7376 - val_accuracy: 0.0000e+00
Epoch 2/100
23/23 [==============================] - 2s 85ms/step - loss: 1.4285 - accuracy: 0.4139 - val_loss: 5.0240 - val_accuracy: 0.0000e+00
Epoch 3/100
23/23 [==============================] - 2s 81ms/step - loss: 1.3112 - accuracy: 0.4710 - val_loss: 4.9924 - val_accuracy: 0.0000e+00
Epoch 4/100
23/23 [==============================] - 2s 82ms/step - loss: 1.1648 - accuracy: 0.5153 - val_loss: 5.4192 - val_accuracy: 0.0000e+00
Epoch 5/100
23/23 [==============================] - 2s 94ms/step - loss: 1.0209 - accuracy: 0.6115 - val_loss: 5.3265 - val_accuracy: 0.0000e+00
Epoch 6/100
23/23 [==============================] - 2s 96ms/step - loss: 0.8981 - accuracy: 0.6712 - val_loss: 6.0114 - val_accuracy: 0.0000e+00
Epoch 7/100
23/23 [==============================] - 2s 94ms/step - loss: 0.7956 - accuracy: 0.7094 - val_loss: 5.7224 - val

In [14]:
x_train_12, x_test_12, y_train_12, y_test_12 = prepare_data(12, integer_encoded)
model_12 = test_model(12,x_train_12, x_test_12, y_train_12, y_test_12)
#4.41

Epoch 1/100
34/34 [==============================] - 5s 90ms/step - loss: 2.2051 - accuracy: 0.2224 - val_loss: 5.1267 - val_accuracy: 0.0000e+00
Epoch 2/100
34/34 [==============================] - 3s 97ms/step - loss: 2.0091 - accuracy: 0.2710 - val_loss: 6.3241 - val_accuracy: 0.0000e+00
Epoch 3/100
34/34 [==============================] - 3s 84ms/step - loss: 1.8624 - accuracy: 0.3323 - val_loss: 6.9145 - val_accuracy: 0.0000e+00
Epoch 4/100
34/34 [==============================] - 3s 87ms/step - loss: 1.6427 - accuracy: 0.4195 - val_loss: 7.1952 - val_accuracy: 0.0000e+00
Epoch 5/100
34/34 [==============================] - 3s 81ms/step - loss: 1.4541 - accuracy: 0.4932 - val_loss: 7.3790 - val_accuracy: 0.0000e+00
Epoch 6/100
34/34 [==============================] - 3s 84ms/step - loss: 1.2919 - accuracy: 0.5490 - val_loss: 7.9664 - val_accuracy: 0.0000e+00
Epoch 7/100
34/34 [==============================] - 3s 85ms/step - loss: 1.1391 - accuracy: 0.6082 - val_loss: 7.7300 - val

In [15]:
x_train_17, x_test_17, y_train_17, y_test_17 = prepare_data(17, integer_encoded)
model_17 = test_model(17,x_train_17, x_test_17, y_train_17, y_test_17)
#5.26

Epoch 1/100
40/40 [==============================] - 4s 83ms/step - loss: 2.4952 - accuracy: 0.1941 - val_loss: 6.0699 - val_accuracy: 0.0000e+00
Epoch 2/100
40/40 [==============================] - 3s 79ms/step - loss: 2.2837 - accuracy: 0.2141 - val_loss: 6.9389 - val_accuracy: 0.0000e+00
Epoch 3/100
40/40 [==============================] - 3s 77ms/step - loss: 2.1927 - accuracy: 0.2516 - val_loss: 7.3488 - val_accuracy: 0.0000e+00
Epoch 4/100
40/40 [==============================] - 3s 81ms/step - loss: 2.0224 - accuracy: 0.3109 - val_loss: 7.7217 - val_accuracy: 0.0000e+00
Epoch 5/100
40/40 [==============================] - 3s 83ms/step - loss: 1.7745 - accuracy: 0.4058 - val_loss: 8.1659 - val_accuracy: 0.0000e+00
Epoch 6/100
40/40 [==============================] - 3s 84ms/step - loss: 1.5569 - accuracy: 0.4750 - val_loss: 8.5710 - val_accuracy: 0.0000e+00
Epoch 7/100
40/40 [==============================] - 3s 78ms/step - loss: 1.3722 - accuracy: 0.5525 - val_loss: 8.7847 - val